In [136]:
def delete_punc(s):
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    return regex.sub('', s)

In [137]:
def to_normal(input_data):
    import pymorphy2
    morph = pymorphy2.MorphAnalyzer()
    result = pd.DataFrame(columns=['Raw', 'Normalized', 'Tags'])
    input_data = input_data.split()
    for i in input_data:
        parsed = morph.parse(i)
        result = result.append({'Raw' : i, 'Normalized' : parsed[0].normal_form, 'Tags' : parsed[0].tag}, ignore_index=True)
    return result

In [138]:
def get_lines(body_task):
    fileObj = body_task
    result = []
    for line in fileObj.split('\n'):
        result.append(line)
    return result

In [139]:
# удаление специальных символов из начала и конца

def get_normal_form_of_prefix_line(lines):
    special_symb = '!@#$%^&*"":* \t'
    result = []
    for line in lines:
        uk = 0;
        for c in line:
            if (c in special_symb):
                uk += 1
            else:
                break
        while (len(line) > 0 and line[-1] in special_symb):
            line = line[:len(line) - 1]
        if (uk != len(line)):
            result.append(line[uk:])
    return result


In [140]:
import datetime

def check_is_date(string):  # xx.xx.xxxx считаем что даты полные, потом можно допилить
    mn_arr = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    digits = '0123456789'
    count = 0
    for i in range(len(string)):
        if (i == 2 or i == 5):
            if (string[i] == '.' or string[i] == '/' or string[i] == '\\' or string[i] == '-'):
                count += 1
        else:
            if (digits.find(string[i]) != -1):
                count += 1
    if (count != len(string)):
        return False
    s = string
    day = int(s[0] + s[1])
    month = int(s[3] + s[4])
    year = int(s[6] + s[7] + s[8] + s[9])
    if (year < 2019):
        return False
    if (year > 2099):
        return False
    if (month > 12):
        return False
    if (month != 2):
        if (mn_arr[month - 1] < day):
            return False
        else:
            return True
    if (month == 2):
        if (year % 4 == 0):
            if (mn_arr[month - 1] + 1 < day):
                return False
            else:
                return True
        else:
            if (mn_arr[month - 1] < day):
                return False
            else:
                return True
    return False;
    
    
    

In [141]:
def less_date(a, b): # true if a <= b
    day_a = int(a[0] + a[1])
    month_a = int(a[3] + a[4])
    year_a = int(a[6] + a[7] + a[8] + a[9])
    
    day_b = int(b[0] + b[1])
    month_b = int(b[3] + b[4])
    year_b = int(b[6] + b[7] + b[8] + b[9])
    if (year_a < year_b):
        return True
    if (year_a > year_b):
        return False
    if (month_a < month_b):
        return True
    if (month_a > month_b):
        return False
    return a <= b;

In [142]:
def get_tasks(words_example, lines):
    words = words_example
    words.append(":")
    digits = '0123456789'
    word_of_theme = ['Приказ', 'приказ', 'заявление', 'Заявление', 'уведомление', 'Уведомление', 'Напоминание', 'напоминание']
    theme = '';
    date = '00.00.0000'
    next_line = False
    tasks = []
    for line in lines:
        for word in word_of_theme:
                if (line.find(word) != -1 and theme == ''):
                    theme = word
        if (next_line):
            tasks.append(line)
            next_line = False
            continue
        if (len(line) > 3):
            if ((line[0] in digits) and ((line[1] == '.') or (line[1] == ')'))):
                tasks.append(line)
            elif ((line[0] in digits) and (line[1] in digits) and ((line[2] == '.') or (line[2] == ')'))):
                tasks.append(line)
            else:
                min_index = len(line)
                max_index = -1
                for word in words:
                    if (line.find(word) != -1):
                        index = line.find(word)
                        if (min_index > index):
                            max_index = index + len(word)
                            min_index = index
                if (max_index == -1):
                    continue
                if (max_index + 5 > len(line)):
                    next_line = True
                else:
                    tasks.append(line[min_index:])
    #if (theme != ''):
    #    tasks.insert(0, theme) тут могда быть ваша тема
    return tasks

In [143]:
def get_date(lines):
    date = '00.00.0000'
    for line in lines:
        for i in range(0, len(line) - 9):
            if (check_is_date(line[i : i + 10])):
                if (less_date(date, line[i : i + 10])):
                    date = line[i : i + 10]
    return date

In [144]:
def main():
    global re
    global string
    global codecs
    global pd
    import re
    import string
    import codecs
    import pandas as pd
    body_task = 'приказ сделать \n \n 1) flex \n 2) kek \n 3) dura_lex_sed_lex \n \n \n 01/01/2020'
    input_data = body_task
    input_data = delete_punc(input_data)
    normalized = to_normal(input_data)
    
    infinities = []
    for i in normalized.index:
        if ('INFN' in normalized.loc[i, 'Tags']) or ('impr' in normalized.loc[i, 'Tags']):
            infinities.append(normalized.loc[i, 'Raw'])
            
    lines = get_lines(body_task)
    lines = get_normal_form_of_prefix_line(lines)
    tasks = get_tasks(infinities, lines)
    deadline = get_date(lines)
    return (tasks, deadline)

In [145]:
main()

(['', '1) flex', '2) kek', '3) dura_lex_sed_lex'], '01/01/2020')


1) flex
2) kek
3) dura_lex_sed_lex
01/01/2020


True